In [4]:
from conductor.client.ai.orchestrator import AIOrchestrator
from conductor.client.ai.configuration import AIConfiguration
from conductor.client.configuration.configuration import Configuration
from conductor.client.configuration.settings.authentication_settings import AuthenticationSettings
from conductor.client.workflow.task.switch_task import SwitchTask

ai_config = AIConfiguration('my_azure_open_ai', 'text-davinci-003', '', 'text-embedding-ada-002', 'pineconedb')
api_config = Configuration(
        authentication_settings=AuthenticationSettings(key_id='0092089d-19ff-4931-b3da-fc093029d0ad',
                                                       key_secret='lzCCrpk5NLi0TYmIDiOfnJWLz5vkxMKn5BoCKIHSddOdYyg3'))

ai_orchestrator = AIOrchestrator(api_configuration=api_config, ai_configuration=ai_config)

result = ai_orchestrator.test_prompt_template('give an evening greeting to ${friend_name}. go: ',
                                                  {'friend_name': 'viren'}, ai_config.llm_provider,
                                                  ai_config.text_complete_model)
print(result)



Good evening Viren! I hope you're having a great day.


# Let's try and run a workflow that uses the LLM Chaining

In [5]:
from conductor.client.workflow.conductor_workflow import ConductorWorkflow
from conductor.client.workflow.task.llm_tasks.utils.prompt import Prompt
from conductor.client.workflow.task.llm_tasks.llm_text_complete import LlmTextComplete
from conductor.client.workflow.task.simple_task import SimpleTask
from conductor.client.http.models import Task, TaskResult

def getName(task: Task) -> str:
    return 'Steve Jobs'

prompt = Prompt(name='say_hi_to_friend', variables={'friend_name': '${get_friend_name_ref.output.result}'})
workflow = ConductorWorkflow(name='say_hi_to_the_friend')
workflow >> SimpleTask(task_def_name='get_friend_name', task_reference_name='get_friend_name_ref', execute_fn=getName) >> LlmTextComplete('say_hi', 'say_hi_ref', ai_config.llm_provider, ai_config.text_complete_model, prompt=prompt) 
workflow.output_parameters = {'greetings': '${say_hi_ref.output.result}'}

# When running from the notebook
# task_executors = {'get_friend_name_ref': getName}

wf_result = ai_orchestrator.execute_workflow(workflow=workflow, wait_for_seconds=5)
print(wf_result.output)

TypeError: __init__() missing 1 required positional argument: 'executor'